In [1]:
import torchvision
from torchvision import transforms
import torch
from torch import nn
from torch.utils.data import DataLoader


In [2]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


In [116]:

device = "cuda:1"
model_classifier0_features1 = NeuralNetwork().to(device)
model_classifier0_features1.load_state_dict(torch.load(f'Fase_0.pth')) 


<All keys matched successfully>

In [117]:
import torch

# Cargar los estados de los modelos
estado_fase_0 = torch.load('Fase_0.pth')
estado_fase_1 = torch.load('Fase_1.pth')


In [118]:
print(estado_fase_1["linear_relu_stack.4.weight"])
print(estado_fase_1["linear_relu_stack.4.bias"])

tensor([[ 0.0113, -0.0353, -0.0856,  ...,  0.0238, -0.0287, -0.0413],
        [ 0.0071,  0.0568, -0.0234,  ...,  0.0192, -0.0632, -0.0044],
        [ 0.0435, -0.0177,  0.0493,  ..., -0.0004,  0.0049, -0.0310],
        ...,
        [ 0.0154,  0.0505,  0.0146,  ..., -0.0349,  0.0459, -0.0251],
        [ 0.0335,  0.0765, -0.0172,  ...,  0.0092, -0.0763, -0.0147],
        [ 0.0238,  0.0163,  0.0683,  ..., -0.0380,  0.0346,  0.0103]],
       device='cuda:0')
tensor([-0.0063,  0.1980,  0.0318, -0.0573,  0.0794,  0.0773, -0.0235,  0.0703,
        -0.1792, -0.0809], device='cuda:0')


In [119]:

# Transferir el clasificador de Fase_0 a Fase_1
estado_fase_1["linear_relu_stack.4.weight"][0:5, :] = estado_fase_0["linear_relu_stack.4.weight"][0:5, :]
estado_fase_1["linear_relu_stack.4.bias"][0:5] = estado_fase_0["linear_relu_stack.4.bias"][0:5]


In [120]:

# Crear el modelo Fase_1 y cargar sus estados actualizados
model_classifier0_features1 = NeuralNetwork().to(device)
model_classifier0_features1.load_state_dict(estado_fase_1)



<All keys matched successfully>

In [67]:
def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    tarjet_prediction = []  # Lista para almacenar las etiquetas reales y predicciones

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

            # Guardar la etiqueta real y la predicción en la lista de tuplas
            tarjet_prediction.extend(list(zip(y.cpu().numpy(), pred.argmax(1).cpu().numpy())))

    test_loss /= num_batches
    correct /= size
    log_accuracy_loss.append((100*correct, test_loss))
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    with open(f'preditions_classifier_changed.txt', 'w') as archivo:
            # Escribe el valor de la variable en el archivo
            archivo.write(str(tarjet_prediction))
    print(f'El valor prediciones se ha guardado en el archivo.txt')
    

In [69]:
eval_data = torchvision.datasets.MNIST(
    root="../data",
    train=False,
    download=True,
    transform=transforms.ToTensor()
)
eval_dataloader = DataLoader(eval_data, batch_size=10000, shuffle=True)
print("Se cargaron los datos correctamente")

Se cargaron los datos correctamente


In [121]:
learning_rate = 1e-3
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_classifier0_features1.parameters(), lr=learning_rate)
epochs = 1
log_accuracy_loss = []
test_loop(eval_dataloader, model_classifier0_features1, loss_fn)


Test Error: 
 Accuracy: 50.3%, Avg loss: 1.233572 

El valor prediciones se ha guardado en el archivo.txt
